In [4]:
#TRAINING

In [5]:
import matplotlib.pyplot as plt  
import numpy as np
import cv2
import os
from skimage.feature import hog

In [6]:
def undesired_objects (image):
    image = image.astype('uint8')
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(image, connectivity=4)
    sizes = stats[:, -1]

    max_label = 1
    max_size = sizes[1]
    for i in range(2, nb_components):
        if sizes[i] > max_size:
            max_label = i
            max_size = sizes[i]

    img2 = np.zeros(output.shape)
    img2[output == max_label] = 255
#     cv2.imshow("Biggest component", img2)
#     cv2.waitKey()
#     cv2.destroyAllWindows()
    return img2

In [7]:
labels = []
hogs = []
h2 = []
h3 = []

In [8]:
for xyz in range (0,1400):
    img = cv2.imread("open/file_" + str(xyz) + ".jpg")
    

    hsvb = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    black_lower=np.array([0,0,0],np.uint8)
    black_upper=np.array([255,255,20],np.uint8)
    black=cv2.inRange(hsvb,black_lower,black_upper)


    blackc = cv2.cvtColor(black,cv2.COLOR_GRAY2BGR)
    blackcomp = np.hstack((img,blackc))
    
    kernel = np.ones((5, 5), np.uint8)
    img_dilation = cv2.dilate(black, kernel, iterations=2)
    img_erosion = cv2.erode(img_dilation, kernel, iterations=2)
    
    
    hand_img = undesired_objects(img_erosion)
    # print(type(hand_img))
    yellowY, yellowX = np.where(hand_img==255)

    top, bottom = min(yellowY), max(yellowY)
    left, right = min(yellowX), max(yellowX)
    # print(top,bottom,left,right)

    # Extract Region of Interest from unblurred original
    ROI = hand_img[top-50:bottom+50, left - 50:right+50]
    
    roi_org = img[top-50:bottom+50,left-50:right+50]
#     print(xyz)
#     print(roi_org.shape)
    if(roi_org.shape[1] == 0 or roi_org.shape[0] == 0):
        continue
    resized_img = cv2.resize(roi_org, (64, 128))
    fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2), visualize=True, multichannel=True)
    hogs.append(fd)
    h2.append(fd)
    labels.append(1)

C:\Users\Umer Mukhtar\AppData\Local\Temp\ipykernel_4344\1895620606.py:37: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2), visualize=True, multichannel=True)


In [9]:
for xyz in range (0,1400):
    img = cv2.imread("close/file_" + str(xyz) + ".jpg")
    

    hsvb = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    black_lower=np.array([0,0,0],np.uint8)
    black_upper=np.array([255,255,20],np.uint8)
    black=cv2.inRange(hsvb,black_lower,black_upper)


    blackc = cv2.cvtColor(black,cv2.COLOR_GRAY2BGR)
    blackcomp = np.hstack((img,blackc))
    
    kernel = np.ones((5, 5), np.uint8)
    img_dilation = cv2.dilate(black, kernel, iterations=2)
    img_erosion = cv2.erode(img_dilation, kernel, iterations=2)
    
    
    hand_img = undesired_objects(img_erosion)
    # print(type(hand_img))
    yellowY, yellowX = np.where(hand_img==255)

    top, bottom = min(yellowY), max(yellowY)
    left, right = min(yellowX), max(yellowX)
    # print(top,bottom,left,right)

    # Extract Region of Interest from unblurred original
    ROI = hand_img[top-50:bottom+50, left - 50:right+50]
    
    roi_org = img[top-50:bottom+50,left-50:right+50]
#     print(xyz)
#     print(roi_org.shape)
    if(roi_org.shape[1] == 0 or roi_org.shape[0] == 0):
        continue
    resized_img = cv2.resize(roi_org, (64, 128))
    fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2), visualize=True, multichannel=True)
    hogs.append(fd)
    h3.append(fd)
    labels.append(2)

C:\Users\Umer Mukhtar\AppData\Local\Temp\ipykernel_4344\3578927075.py:37: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2), visualize=True, multichannel=True)


In [10]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators = 50, max_depth=None, min_samples_split=2, random_state=0)

classifier.fit(hogs,labels)


print(classifier.score(hogs,labels))

1.0


1.0

In [11]:
import pickle

# save
with open('model.pkl','wb') as f:
    pickle.dump(classifier,f)



In [45]:
with open('model.pkl', 'rb') as f:
    clf2 = pickle.load(f)

arr = clf2.predict(h2)
arr

array([1, 1, 1, ..., 1, 1, 1])